In [1]:
import setup

setup.init_django()

In [2]:
from rag import (
    db as rag_db,
    engines as rag_engines,
    settings as rag_settings,
    updaters as rag_updaters,
    sync as rag_sync, 
    patches as rag_patches,
)


In [3]:
from typing import Optional, Union
from sqlalchemy import create_engine, text

In [ ]:
rag_settings.init()
rag_db.init_vector_db()
rag_sync.full_sync()

In [4]:
employee_index = rag_engines.get_model_index("Employee")
product_index = rag_engines.get_model_index("Product")
semantic_query_engine = rag_engines.get_semantic_query_engine("Product")  
sql_query_engine = rag_engines.get_sql_query_engine()

In [5]:
print(rag_settings.VECTOR_DB_NAME, rag_settings.VECTOR_DB_TABLE_NAMES)

vector_db ['employeeroles', 'employees', 'producttypes', 'products', 'inventoryitems', 'productinventoryrequirements']


In [6]:
from llama_index.core.tools import QueryEngineTool

vector_tool = QueryEngineTool.from_defaults(
    query_engine=semantic_query_engine,
    description=(
        "Useful for answering semantic questions about coffee shop operations, "
        "including products, employees, inventory, and recipes"
    ),
)


In [7]:
sql_tool = QueryEngineTool.from_defaults(
    query_engine=sql_query_engine,
    description=(
        "Useful for translating a natural language query into a SQL query over "
        "tables containing:EmployeeRole, Employee, ProductType, Product, InventoryItem, ProductInventoryRequirement."
    ),
)

In [8]:
query_engine = rag_patches.MySQLAutoVectorQueryEngine(
    sql_tool,
    vector_tool,
)

In [9]:
response = query_engine.query(
    "how mush is an late?"
)

Querying other query engine: The question 'how mush is an late?' seems to be related to coffee shop operations, specifically about products (like a 'late' which could be a typo for 'latte').


In [10]:
from IPython.display import Markdown, display

display(Markdown(response.response))

The price is 4.50.

In [11]:
response = query_engine.query(
    "give me the list of all employee"
)

Querying SQL database: The question asks for a list of employees, and choice (1) mentions 'Employee' and 'EmployeeRole,' which are relevant to the query.
SQL query: SELECT coffeshop_employee.name, coffeshop_employee.hire_date, coffeshop_employee.hourly_rate, coffeshop_employee.is_active, coffeshop_employeerole.name AS role_name
FROM coffeshop_employee
JOIN coffeshop_employeerole ON coffeshop_employee.role_id = coffeshop_employeerole.id
ORDER BY coffeshop_employee.name;
SQL response: Here is the list of all employees:

1. **David Wilson**
   - Hire Date: April 26, 2024
   - Hourly Rate: $18.00
   - Role: Inventory Specialist
   - Status: Active

2. **Emily Davis**
   - Hire Date: October 17, 2024
   - Hourly Rate: $13.50
   - Role: Waiter
   - Status: Active

3. **Harry Smith**
   - Hire Date: March 5, 2022
   - Hourly Rate: $35.00
   - Role: Manager
   - Status: Active

4. **Jane Smith**
   - Hire Date: April 2, 2024
   - Hourly Rate: $15.00
   - Role: Barista
   - Status: Active

5. *

In [12]:
from IPython.display import Markdown, display

display(Markdown(response.response))

Here is the list of all employees:

1. **David Wilson**
   - Hire Date: April 26, 2024
   - Hourly Rate: $18.00
   - Role: Inventory Specialist
   - Status: Active

2. **Emily Davis**
   - Hire Date: October 17, 2024
   - Hourly Rate: $13.50
   - Role: Waiter
   - Status: Active

3. **Harry Smith**
   - Hire Date: March 5, 2022
   - Hourly Rate: $35.00
   - Role: Manager
   - Status: Active

4. **Jane Smith**
   - Hire Date: April 2, 2024
   - Hourly Rate: $15.00
   - Role: Barista
   - Status: Active

5. **Jasmine Lee**
   - Hire Date: February 26, 2025
   - Hourly Rate: $15.00
   - Role: Kitchen Staff
   - Status: Active

6. **John Doe**
   - Hire Date: March 26, 2024
   - Hourly Rate: $15.00
   - Role: Barista
   - Status: Active

7. **Michael Johnson**
   - Hire Date: May 26, 2024
   - Hourly Rate: $16.00
   - Role: Kitchen Staff
   - Status: Active

8. **Robert Brown**
   - Hire Date: January 7, 2025
   - Hourly Rate: $14.00
   - Role: Cashier
   - Status: Active

In [13]:
response = query_engine.query(
    "لیست تمام کارکنان"
)

Querying SQL database: The query 'لیست تمام کارکنان' translates to 'list all employees,' which is relevant to translating a natural language query into a SQL query over tables containing EmployeeRole and Employee.
SQL query: SELECT
    coffeshop_employee.id,
    coffeshop_employee.name,
    coffeshop_employee.hire_date,
    coffeshop_employee.hourly_rate,
    coffeshop_employee.is_active,
    coffeshop_employeerole.name AS role_name
FROM
    coffeshop_employee
JOIN
    coffeshop_employeerole
ON
    coffeshop_employee.role_id = coffeshop_employeerole.id
ORDER BY
    coffeshop_employee.name;
SQL response: لیست تمام کارکنان کافی شاپ به شرح زیر است:

1. **David Wilson**
   - شناسه: 12
   - تاریخ استخدام: 2024-04-26
   - حقوق ساعتی: 18.00
   - وضعیت: فعال
   - نقش: Inventory Specialist

2. **Emily Davis**
   - شناسه: 9
   - تاریخ استخدام: 2024-10-17
   - حقوق ساعتی: 13.50
   - وضعیت: فعال
   - نقش: Waiter

3. **Harry Smith**
   - شناسه: 5
   - تاریخ استخدام: 2022-03-05
   - حقوق ساعتی: 35.0

In [14]:
from IPython.display import Markdown, display

display(Markdown(response.response))

لیست تمام کارکنان کافی شاپ به شرح زیر است:

1. **David Wilson**
   - شناسه: 12
   - تاریخ استخدام: 2024-04-26
   - حقوق ساعتی: 18.00
   - وضعیت: فعال
   - نقش: Inventory Specialist

2. **Emily Davis**
   - شناسه: 9
   - تاریخ استخدام: 2024-10-17
   - حقوق ساعتی: 13.50
   - وضعیت: فعال
   - نقش: Waiter

3. **Harry Smith**
   - شناسه: 5
   - تاریخ استخدام: 2022-03-05
   - حقوق ساعتی: 35.00
   - وضعیت: فعال
   - نقش: Manager

4. **Jane Smith**
   - شناسه: 7
   - تاریخ استخدام: 2024-04-02
   - حقوق ساعتی: 15.00
   - وضعیت: فعال
   - نقش: Barista

5. **Jasmine Lee**
   - شناسه: 11
   - تاریخ استخدام: 2025-02-26
   - حقوق ساعتی: 15.00
   - وضعیت: فعال


In [19]:
response = query_engine.query(
    "averge price of all products"
)

Querying SQL database: The first choice mentions 'Product' and 'ProductType', which are relevant to determining the average price of all products.
SQL query: SELECT AVG(coffeshop_product.price) AS average_price
FROM coffeshop_product;
SQL response: The average price of all products in the coffee shop is $3.56.
Transformed query given SQL response: None


In [20]:
from IPython.display import Markdown, display

display(Markdown(response.response))

The average price of all products in the coffee shop is $3.56.